In [1]:
from os import listdir
import tensorflow as tf
from numpy import array
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.utils import to_categorical
from keras.layers import Embedding, TimeDistributed, RepeatVector, LSTM, concatenate , Input, Reshape, Dense, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
import numpy as np
from IPython.display import Image, display

In [2]:
# Load the images
images = []
all_filenames = listdir('images/')
all_filenames.sort()
for filename in all_filenames:
    #display(Image(filename='images/'+filename))
    images.append(img_to_array(load_img('images/'+filename, target_size=(299, 299))))
images = np.array(images, dtype=float)
images = preprocess_input(images)

In [3]:
# Run the images through inception-resnet
IR2 = InceptionResNetV2(weights='imagenet', include_top=False)
features = IR2.predict(images)

In [4]:
# Number of words to cap input for HTML at (this could change)
max_caption_len = 100

# Use this tokenizer object to split the HTML strings into words
tokenizer = Tokenizer(filters='', split=" ", lower=False)

# Load the HTML files
X = []
all_filenames = listdir('html/')
all_filenames.sort()
for filename in all_filenames:
    file = open('html/'+filename, 'r')
    text = file.read()
    file.close()
    X.append(text)

# Use the tokenizer to divide the text
tokenizer.fit_on_texts(X)

# Add +1 to leave space for empty words
vocab_size = len(tokenizer.word_index) + 1

# Translate each word in text file to the matching vocabulary index
sequences = tokenizer.texts_to_sequences(X)

# The length of the longest HTML document
max_length = max(len(s) for s in sequences)

In [5]:
# Intialize our final input to the model
X, X2, Y = list(), list(), list()
for img_no, seq in enumerate(sequences):
    for i in range(1, len(seq)):
        # Add the entire sequence to the input and only keep the next word for the output
        in_seq, out_seq = seq[:i], seq[i]
        # If the sentence is shorter than max_length, fill it up with empty words
        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
        # Map the output to one-hot encoding
        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
        # Add and image corresponding to the HTML file
        X2.append(features[img_no])
        # Cut the input sentence to 100 tokens, and add it to the input data
        X.append(in_seq[-100:])
        Y.append(out_seq)
X, X2, Y = np.array(X), np.array(X2), np.array(Y)

In [6]:
# Model Creation

# Input 1
language_input = Input(shape=(max_caption_len,))
language_model = Embedding(vocab_size, 200, input_length=max_caption_len)(language_input)
language_model = LSTM(256, return_sequences=True)(language_model)
language_model = LSTM(256, return_sequences=True)(language_model)
language_model = TimeDistributed(Dense(128, activation='relu'))(language_model)

# Input 2
image_features = Input(shape=(8, 8, 1536,))
image_flat = Flatten()(image_features)
image_flat = Dense(128, activation='relu')(image_flat)
ir2_out = RepeatVector(max_caption_len)(image_flat)

# Concatenate two input sequences
decoder = concatenate([ir2_out, language_model])
decoder = LSTM(512, return_sequences=False)(decoder)
decoder_output = Dense(vocab_size, activation='softmax')(decoder)

# Compile the model
model = Model(inputs=[image_features, language_input], outputs=decoder_output)
model.summary()
model.compile(metrics=['accuracy'], loss='categorical_crossentropy', optimizer='rmsprop')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 8, 8, 1536)] 0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 200)     87200       input_2[0][0]                    
__________________________________________________________________________________________________
flatten (Flatten)               (None, 98304)        0           input_3[0][0]                    
______________________________________________________________________________________________

In [8]:
# Train the neural network
model.fit([X2, X], Y, batch_size=64, shuffle=False, epochs=550)

Epoch 1/550
37/37 [==============================] - 2s 54ms/step - loss: 5.5590 - accuracy: 0.0165
Epoch 2/550
37/37 [==============================] - 2s 55ms/step - loss: 5.5377 - accuracy: 0.0212
Epoch 3/550
37/37 [==============================] - 2s 55ms/step - loss: 5.4850 - accuracy: 0.0212
Epoch 4/550
37/37 [==============================] - 2s 55ms/step - loss: 5.4705 - accuracy: 0.0234
Epoch 5/550
37/37 [==============================] - 2s 55ms/step - loss: 5.4362 - accuracy: 0.0143
Epoch 6/550
37/37 [==============================] - 2s 55ms/step - loss: 5.4111 - accuracy: 0.0217
Epoch 7/550
37/37 [==============================] - 2s 54ms/step - loss: 5.3895 - accuracy: 0.0182
Epoch 8/550
37/37 [==============================] - 2s 54ms/step - loss: 5.3422 - accuracy: 0.0191
Epoch 9/550
37/37 [==============================] - 2s 55ms/step - loss: 5.2890 - accuracy: 0.0230
Epoch 10/550
37/37 [==============================] - 2s 54ms/step - loss: 5.2510 - accuracy: 0.0221

37/37 [==============================] - 2s 54ms/step - loss: 3.5191 - accuracy: 0.1344
Epoch 83/550
37/37 [==============================] - 2s 54ms/step - loss: 3.5437 - accuracy: 0.1383
Epoch 84/550
37/37 [==============================] - 2s 54ms/step - loss: 3.4875 - accuracy: 0.1392
Epoch 85/550
37/37 [==============================] - 2s 55ms/step - loss: 3.3425 - accuracy: 0.1544
Epoch 86/550
37/37 [==============================] - 2s 54ms/step - loss: 3.4024 - accuracy: 0.1561
Epoch 87/550
37/37 [==============================] - 2s 54ms/step - loss: 3.3385 - accuracy: 0.1561
Epoch 88/550
37/37 [==============================] - 2s 54ms/step - loss: 3.2906 - accuracy: 0.1622
Epoch 89/550
37/37 [==============================] - 2s 55ms/step - loss: 3.2643 - accuracy: 0.1687
Epoch 90/550
37/37 [==============================] - 2s 55ms/step - loss: 3.2294 - accuracy: 0.1739
Epoch 91/550
37/37 [==============================] - 2s 55ms/step - loss: 3.1936 - accuracy: 0.1808
Epo

37/37 [==============================] - 2s 59ms/step - loss: 0.2592 - accuracy: 0.9402
Epoch 163/550
37/37 [==============================] - 2s 59ms/step - loss: 0.2361 - accuracy: 0.9514
Epoch 164/550
37/37 [==============================] - 2s 60ms/step - loss: 0.2763 - accuracy: 0.9324
Epoch 165/550
37/37 [==============================] - 2s 60ms/step - loss: 0.2520 - accuracy: 0.9410
Epoch 166/550
37/37 [==============================] - 2s 60ms/step - loss: 0.3188 - accuracy: 0.9328
Epoch 167/550
37/37 [==============================] - 2s 59ms/step - loss: 0.2003 - accuracy: 0.9549
Epoch 168/550
37/37 [==============================] - 2s 60ms/step - loss: 0.1826 - accuracy: 0.9575
Epoch 169/550
37/37 [==============================] - 2s 60ms/step - loss: 0.1760 - accuracy: 0.9536
Epoch 170/550
37/37 [==============================] - 2s 59ms/step - loss: 0.6842 - accuracy: 0.8990
Epoch 171/550
37/37 [==============================] - 2s 59ms/step - loss: 0.3923 - accuracy: 0

37/37 [==============================] - 2s 56ms/step - loss: 0.0188 - accuracy: 0.9957
Epoch 243/550
37/37 [==============================] - 2s 57ms/step - loss: 0.0369 - accuracy: 0.9896
Epoch 244/550
37/37 [==============================] - 2s 57ms/step - loss: 0.0242 - accuracy: 0.9926
Epoch 245/550
37/37 [==============================] - 2s 55ms/step - loss: 0.0227 - accuracy: 0.9918
Epoch 246/550
37/37 [==============================] - 2s 54ms/step - loss: 0.0205 - accuracy: 0.9935
Epoch 247/550
37/37 [==============================] - 2s 54ms/step - loss: 0.0741 - accuracy: 0.9766
Epoch 248/550
37/37 [==============================] - 2s 54ms/step - loss: 0.0980 - accuracy: 0.9727
Epoch 249/550
37/37 [==============================] - 2s 54ms/step - loss: 0.0407 - accuracy: 0.9870
Epoch 250/550
37/37 [==============================] - 2s 55ms/step - loss: 0.0533 - accuracy: 0.9844
Epoch 251/550
37/37 [==============================] - 2s 55ms/step - loss: 0.0387 - accuracy: 0

37/37 [==============================] - 2s 53ms/step - loss: 0.0086 - accuracy: 0.9970
Epoch 323/550
37/37 [==============================] - 2s 55ms/step - loss: 0.0090 - accuracy: 0.9961
Epoch 324/550
37/37 [==============================] - 2s 53ms/step - loss: 0.0078 - accuracy: 0.9957
Epoch 325/550
37/37 [==============================] - 2s 53ms/step - loss: 0.0096 - accuracy: 0.9952
Epoch 326/550
37/37 [==============================] - 2s 53ms/step - loss: 0.0081 - accuracy: 0.9957
Epoch 327/550
37/37 [==============================] - 2s 53ms/step - loss: 0.0101 - accuracy: 0.9957
Epoch 328/550
37/37 [==============================] - 2s 54ms/step - loss: 0.0144 - accuracy: 0.9948
Epoch 329/550
37/37 [==============================] - 2s 54ms/step - loss: 0.0132 - accuracy: 0.9952
Epoch 330/550
37/37 [==============================] - 2s 53ms/step - loss: 0.0166 - accuracy: 0.9935
Epoch 331/550
37/37 [==============================] - 2s 53ms/step - loss: 0.0076 - accuracy: 0

37/37 [==============================] - 3s 84ms/step - loss: 0.0046 - accuracy: 0.9974
Epoch 403/550
37/37 [==============================] - 3s 82ms/step - loss: 0.0037 - accuracy: 0.9974
Epoch 404/550
37/37 [==============================] - 3s 79ms/step - loss: 0.0036 - accuracy: 0.9974
Epoch 405/550
37/37 [==============================] - 3s 83ms/step - loss: 0.0036 - accuracy: 0.9978
Epoch 406/550
37/37 [==============================] - 3s 84ms/step - loss: 0.0056 - accuracy: 0.9970
Epoch 407/550
37/37 [==============================] - 3s 82ms/step - loss: 0.0071 - accuracy: 0.9965
Epoch 408/550
37/37 [==============================] - 3s 83ms/step - loss: 0.0114 - accuracy: 0.9961
Epoch 409/550
37/37 [==============================] - 3s 82ms/step - loss: 0.0083 - accuracy: 0.9957
Epoch 410/550
37/37 [==============================] - 3s 84ms/step - loss: 0.0081 - accuracy: 0.9970
Epoch 411/550
37/37 [==============================] - 3s 84ms/step - loss: 0.0083 - accuracy: 0

37/37 [==============================] - 3s 85ms/step - loss: 0.0026 - accuracy: 0.9983
Epoch 483/550
37/37 [==============================] - 3s 87ms/step - loss: 0.0026 - accuracy: 0.9983
Epoch 484/550
37/37 [==============================] - 3s 83ms/step - loss: 0.0026 - accuracy: 0.9983
Epoch 485/550
37/37 [==============================] - 3s 83ms/step - loss: 0.0026 - accuracy: 0.9983
Epoch 486/550
37/37 [==============================] - 3s 86ms/step - loss: 0.0026 - accuracy: 0.9983
Epoch 487/550
37/37 [==============================] - 3s 86ms/step - loss: 0.0026 - accuracy: 0.9983
Epoch 488/550
37/37 [==============================] - 3s 85ms/step - loss: 0.0026 - accuracy: 0.9983
Epoch 489/550
37/37 [==============================] - 3s 86ms/step - loss: 0.0026 - accuracy: 0.9983
Epoch 490/550
37/37 [==============================] - 3s 86ms/step - loss: 0.0026 - accuracy: 0.9978
Epoch 491/550
37/37 [==============================] - 3s 88ms/step - loss: 0.0026 - accuracy: 0